In [1]:
import pandas as pd

In [2]:

# Use forward slashes in the file path
df = pd.read_csv("D:/1. Courses/Python/Projects/Song Recommendation system/spotify_millsongdata.csv")


In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Andy Williams,The Impossible Dream,To dream the impossible dream \r\nTo fight th...
1,Lenny Kravitz,Superlove,"I want you \r\nAll of you \r\nCan't leave, b..."
2,Reba Mcentire,I Wouldn't Wanna Be You,You don't have a clue \r\nYou're so far gone ...
3,Nick Cave,Fable Of The Brown Ape,Farmer Emmerich went into his barn \r\nAnd fo...
4,Pearl Jam,Breakerfall,There's a girl on a ledge she's got nowhere to...
5,Uncle Kracker,Follow Me,You don't know how you met me \r\nYou don't k...
6,Steely Dan,I.G.Y.,Standing tough under stars and stripes \r\nWe...
7,Omd,If You Leave,"If you leave, don't leave now \r\nPlease don'..."
8,Miley Cyrus,Dancing,"Here I am, dancing in the wind \r\nMy back to..."
9,Van Morrison,Glad Tidings,And they'll lay you down low in the easy \r\n...


In [10]:
df['text'][0]

"To dream the impossible dream  \r\nTo fight the unbeatable foe  \r\nTo bear with unbearable sorrow  \r\nTo run where the brave dare not go  \r\n  \r\nTo right the unrightable wrong  \r\nTo love pure and chaste from afar  \r\nTo try when your arms are too weary  \r\nTo reach the unreachable star  \r\n  \r\nThis is my quest, to follow that star,  \r\nNo matter how hopeless, no matter how far  \r\nTo fight for the right without question or cause  \r\nTo be willing to march into hell for a heavenly cause  \r\n  \r\nAnd I know if I'll only be true to this glorious quest  \r\nThat my heart will lie peaceful and calm when I'm laid to my rest  \r\n  \r\nAnd the world will be better for this  \r\nThat one man scorned and covered with scars  \r\nStill strove with his last ounce of courage  \r\nTo fight the unbeatable foe, to reach the unreachable star\r\n\r\n"

In [11]:
df.shape

(5000, 3)

# **Text Cleaning/ Text Preprocessing**

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.00000000e+00, 2.61597793e-04, 5.47680992e-02, ...,
       2.09487141e-02, 3.55301856e-02, 3.69986918e-02])

In [23]:
df[df['song'] == 'The Impossible Dream']

,artist,song,text
0,Andy Williams,The Impossible Dream,to dream the imposs dream to fight the unbeat ...


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [25]:
def recommendation(song_title):
    # Find the index of the song
    song_row = df[df['song'] == song_title]
    
    if song_row.empty:
        print(f"Song '{song_title}' not found in the DataFrame.")
        return None
    
    idx = song_row.index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    recommended_songs = []
    for i in distances[1:6]:  # Get top 5 recommendations (excluding the song itself)
        recommended_songs.append(df.iloc[i[0]]['song'])  # Adjust based on the correct column name
    
    return recommended_songs

# Test the function
print(recommendation('The Impossible Dream'))


["Can't Fight This Feeling", 'A Matter Of Love', 'Glorious Day', 'Fight', 'Hopeless']


In [26]:
recommendation('The Impossible Dream')

["Can't Fight This Feeling",
 'A Matter Of Love',
 'Glorious Day',
 'Fight',
 'Hopeless']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))